# Notebook for checking that my data is what I expect

In [3]:
from __future__ import division, print_function, absolute_import

import time
import random
import glob
import os.path
import textwrap

from IPython.display import display, HTML, Image, Latex

%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
plt.rcParams['figure.figsize'] = 8, 8
import seaborn as sns
sns.set_style('white')

import numpy as np
import pandas as pd
import scipy.optimize
import scipy.spatial
import scipy.stats
import cv2
from tf import transformations
import inspect

In [3]:

data = np.load("../data/marker_calibration.npz")
df = pd.DataFrame(data=data["t_rcs"], columns=["x","y","z", "roll","pitch", "yaw"])
df

,x,y,z,roll,pitch,yaw
0,-0.007029,0.018354,-0.010383,0.009594,0.050396,0.024129
1,-0.006394,0.019277,-0.009494,0.009973,0.050712,0.023212
2,-0.006378,0.018236,-0.010283,0.010368,0.050273,0.023989
3,-0.006874,0.018793,-0.009106,0.010941,0.051137,0.023820
4,-0.006564,0.019117,-0.009241,0.009808,0.051090,0.023317


In [4]:
data = np.load("../data/board_calibration.npz")
df = pd.DataFrame(data=data["t_rcs"], columns=["x","y","z", "roll","pitch", "yaw"])
df

,x,y,z,roll,pitch,yaw
0,-0.013384,0.014086,-0.006880,0.016111,0.051603,0.028777
1,-0.013570,0.014193,-0.006426,0.015043,0.052642,0.028176
2,-0.013661,0.014105,-0.007261,0.014912,0.051095,0.029069
3,-0.013533,0.014703,-0.008155,0.015520,0.049091,0.028422
4,-0.013636,0.014154,-0.008333,0.015693,0.049263,0.029179
5,-0.013224,0.014840,-0.007879,0.015108,0.049958,0.027850


In [16]:
data = np.load("../data/all_markers_10_9_18.npz")
print(len(set(data["bag_files"])))

data = np.load("../data/all_boards_10_9_18.npz")
print(len(set(data["bag_files"])))

17
31


In [10]:
data = np.load("../data/all_boards.npz")
# print(list(data["camera_marker_counts"]).count(6))
# print(list(data["camera_marker_counts"]).count(5))
len(set(data["bag_files"]))

23

In [19]:
a = ['/home/sam/Desktop/calibration_data_10-9-18/markers/27_2018-09-10-12-54-06.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/31_2018-09-10-12-55-19.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/29_2018-09-10-12-54-39.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/24_2018-09-10-12-53-17.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/39_2018-09-10-12-56-59.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/26_2018-09-10-12-53-55.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/40_2018-09-10-12-57-13.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/28_2018-09-10-12-54-26.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/35_2018-09-10-12-56-02.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/25_2018-09-10-12-53-28.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/22_2018-09-10-12-52-46.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/23_2018-09-10-12-52-59.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/37_2018-09-10-12-56-30.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/38_2018-09-10-12-56-42.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/34_2018-09-10-12-55-52.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/36_2018-09-10-12-56-16.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/33_2018-09-10-12-55-41.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/32_2018-09-10-12-55-30.bag', '/home/sam/Desktop/calibration_data_10-9-18/markers/30_2018-09-10-12-54-50.bag']
for bag in sorted(a):
    print(bag.split("/")[-1])

22_2018-09-10-12-52-46.bag
23_2018-09-10-12-52-59.bag
24_2018-09-10-12-53-17.bag
25_2018-09-10-12-53-28.bag
26_2018-09-10-12-53-55.bag
27_2018-09-10-12-54-06.bag
28_2018-09-10-12-54-26.bag
29_2018-09-10-12-54-39.bag
30_2018-09-10-12-54-50.bag
31_2018-09-10-12-55-19.bag
32_2018-09-10-12-55-30.bag
33_2018-09-10-12-55-41.bag
34_2018-09-10-12-55-52.bag
35_2018-09-10-12-56-02.bag
36_2018-09-10-12-56-16.bag
37_2018-09-10-12-56-30.bag
38_2018-09-10-12-56-42.bag
39_2018-09-10-12-56-59.bag
40_2018-09-10-12-57-13.bag
